In [2]:
from datasets import load_dataset
from transformers import pipeline

raw_dataset = load_dataset('cnn_dailymail', '3.0.0')

classifier = pipeline("summarization")

/opt/homebrew/Caskroom/miniconda/base/envs/NLP-summarization/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 132.37it/s]


In [24]:
def show_examples(dataset, num_samples=3, seed=42):
    samples = dataset["train"].shuffle(seed=seed).select(range(num_samples))
        
    for idx, sample in enumerate(samples):
        print(f'sample {idx}: {sample["article"]} \n')
        print(f'highlight {idx}: {sample["highlights"]} \n')
        print('-------')

In [25]:
show_examples(raw_dataset, 1,)

Loading cached shuffled indices for dataset at /Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-516bef66c83f0d37.arrow


sample 0: A protester in Ferguson was arrested during a demonstration on Thursday night - and live-tweeted her entire experience. Brittany Ferrell, a nursing student at the University of Missouri-Saint Louis, was one of 13 people detained by officers in the conflicted Missouri city for 'noise disruption'. The detention has sparked an investigation by the American Civil Liberties Union as lawyers accuse officers of overstretching their powers. Scroll down for video . Arrested: This is Brittany Ferrell, the nursing student and protester who live-tweeted her arrest in Ferguson . Tweeting in handcuffs, she then took her thousands of followers through the uncertain few hours before the nine women and four men eventually learned what they had done wrong and managed to secure their release. She started with the tweet: 'We are all arrested', apparently referring to herself an other members of the advocacy group Millenial Activists United. 'Tweeting while cuffed,' she wrote, adding: 'We were ar